In [1]:
%cd ..

/users/5/dever120/PyGRANSO


# Eigenvalue Optimization

Eigenvalue Optimization taken from: [GRANSO](http://www.timmitchell.com/software/GRANSO/) demo example 4.

## Problem Description

We have $M=A+BXC$,
where the matrices $A\in R^{N,N},B\in R^{N,M}$ and $C\in R^{P,N}$ are given, $X\in R^{M,P}$ is the matrix form optimization variable.

We have the nonconvex, nonsmooth, and constrained optimization problem
$$\min_{X}\max| \mathrm{Im} (\Lambda(A+BXC))|,$$
$$\text{s.t. }\alpha(A+BXC)+\xi \leq 0,$$
where $\mathrm{Im}(\cdot)$ is the imaginary part of complex number, $\xi$ is the stability margin, and $\Lambda(\cdot)$ is the spectrum of a square matrix $\cdot$, and $\alpha(\cdot)$ is the spectral abscissa of a square matrix, i.e., the maximum real part of its eigenvalues.

## Modules Importing
Import all necessary modules and add PyGRANSO src folder to system path.

In [2]:
import time
import torch
from pygranso.pygranso import pygranso
from pygranso.pygransoStruct import pygransoStruct
import scipy.io
from torch import linalg as LA

## Data Initialization 
Specify torch device, and read the data from a provided file.

Use GPU for this problem. If no cuda device available, please set *device = torch.device('cpu')*

In [3]:
device = torch.device('cpu')

file = "/users/5/dever120/PyGRANSO/examples/data/spec_radius_opt_data.mat"
mat = scipy.io.loadmat(file)
mat_struct = mat['sys']
mat_struct = mat_struct[0,0]
# All the user-provided data (vector/matrix/tensor) must be in torch tensor format. 
# As PyTorch tensor is single precision by default, one must explicitly set `dtype=torch.double`.
# Also, please make sure the device of provided torch tensor is the same as opts.torch_device.
A = torch.from_numpy(mat_struct['A']).to(device=device, dtype=torch.double)
B = torch.from_numpy(mat_struct['B']).to(device=device, dtype=torch.double)
C = torch.from_numpy(mat_struct['C']).to(device=device, dtype=torch.double)
p = B.shape[1]
m = C.shape[0]
stability_margin = 1

## Function Set-Up

Encode the optimization variables, and objective and constraint functions.

Note: please strictly follow the format of comb_fn, which will be used in the PyGRANSO main algortihm.

In [4]:
# variables and corresponding dimensions.
var_in = {"X": [p,m] }

def user_fn(X_struct,A,B,C,stability_margin):
    # user defined variable, matirx form. torch tensor
    X = X_struct.X

    # objective function
    M = A + B @ X @ C
    [D, _] = LA.eig(M)
    f = torch.max(D.imag)

    # inequality constraint, matrix form
    ci = pygransoStruct()
    ci.c1 = torch.max(D.real) + stability_margin

    # equality constraint 
    ce = None
    
    return [f, ci, ce]

comb_fn = lambda X_struct : user_fn(X_struct, A, B, C, stability_margin)

## User Options
Specify user-defined options for PyGRANSO

In [5]:
opts = pygransoStruct()
opts.torch_device = device
opts.maxit = 1000
opts.x0 = torch.zeros(p * m, 1).to(device=device, dtype=torch.double)
# print for every 10 iterations. default: 1
opts.print_frequency = 10

## Main Algorithm

In [6]:
start = time.time()
cpu_soln = pygranso(var_spec=var_in, combined_fn=comb_fn, user_opts=opts)
end = time.time()
print("Total Wall Time: {}s".format(end - start))



╔═════ QP SOLVER NOTICE ════════════════════════════════════════════════════════════════════════╗
║  PyGRANSO requires a quadratic program (QP) solver that has a quadprog-compatible interface,  ║
║  the default is osqp. Users may provide their own wrapper for the QP solver.                  ║
║  To disable this notice, set opts.quadprog_info_msg = False                                   ║
╚═══════════════════════════════════════════════════════════════════════════════════════════════╝
═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╗
PyGRANSO: A PyTorch-enabled port of GRANSO with auto-differentiation                                             ║ 
Version 1.2.0                                                                                                    ║ 
Licensed under the AGPLv3, Copyright (C) 2021-2022 Tim Mitchell and Buyun Liang                                  ║ 
═════════════════════════════════════════════

## CUDA Computation

In [7]:
device = torch.device('cuda')

file = "/users/5/dever120/PyGRANSO/examples/data/spec_radius_opt_data.mat"
mat = scipy.io.loadmat(file)
mat_struct = mat['sys']
mat_struct = mat_struct[0,0]
# All the user-provided data (vector/matrix/tensor) must be in torch tensor format. 
# As PyTorch tensor is single precision by default, one must explicitly set `dtype=torch.double`.
# Also, please make sure the device of provided torch tensor is the same as opts.torch_device.
A = torch.from_numpy(mat_struct['A']).to(device=device, dtype=torch.double)
B = torch.from_numpy(mat_struct['B']).to(device=device, dtype=torch.double)
C = torch.from_numpy(mat_struct['C']).to(device=device, dtype=torch.double)
p = B.shape[1]
m = C.shape[0]
stability_margin = 1

In [8]:
# variables and corresponding dimensions.
var_in = {"X": [p,m] }

def user_fn(X_struct,A,B,C,stability_margin):
    # user defined variable, matirx form. torch tensor
    X = X_struct.X

    # objective function
    M = A + B @ X @ C
    [D, _] = LA.eig(M)
    f = torch.max(D.imag)

    # inequality constraint, matrix form
    ci = pygransoStruct()
    ci.c1 = torch.max(D.real) + stability_margin

    # equality constraint 
    ce = None
    
    return [f, ci, ce]

comb_fn = lambda X_struct : user_fn(X_struct, A, B, C, stability_margin)

In [9]:
opts = pygransoStruct()
opts.torch_device = device
opts.maxit = 1000
opts.x0 = torch.zeros(p * m, 1).to(device=device, dtype=torch.double)
# print for every 10 iterations. default: 1
opts.print_frequency = 10

In [10]:
start = time.time()
cuda_soln = pygranso(var_spec=var_in, combined_fn=comb_fn, user_opts=opts)
end = time.time()
print("Total Wall Time: {}s".format(end - start))



╔═════ QP SOLVER NOTICE ════════════════════════════════════════════════════════════════════════╗
║  PyGRANSO requires a quadratic program (QP) solver that has a quadprog-compatible interface,  ║
║  the default is osqp. Users may provide their own wrapper for the QP solver.                  ║
║  To disable this notice, set opts.quadprog_info_msg = False                                   ║
╚═══════════════════════════════════════════════════════════════════════════════════════════════╝
═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╗
PyGRANSO: A PyTorch-enabled port of GRANSO with auto-differentiation                                             ║ 
Version 1.2.0                                                                                                    ║ 
Licensed under the AGPLv3, Copyright (C) 2021-2022 Tim Mitchell and Buyun Liang                                  ║ 
═════════════════════════════════════════════

## Testing

In [15]:
torch.testing.assert_close(
    cpu_soln.final.x,
    cuda_soln.final.x.cpu(),
    atol=1e-2,
    rtol=1e-2,
)

AssertionError: Tensor-likes are not close!

Mismatched elements: 35 / 200 (17.5%)
Greatest absolute difference: 0.024607874336102162 at index (103, 0) (up to 0.01 allowed)
Greatest relative difference: 58.06304384630274 at index (5, 0) (up to 0.01 allowed)

In [18]:
cpu_soln.final.x[:10]

tensor([[-0.0297],
        [ 0.0087],
        [-0.0107],
        [-0.0038],
        [-0.0236],
        [ 0.0134],
        [ 0.0067],
        [-0.0208],
        [-0.0008],
        [-0.0100]], dtype=torch.float64)

In [19]:
cuda_soln.final.x.cpu()[:10]

tensor([[-0.0285],
        [-0.0028],
        [-0.0071],
        [-0.0262],
        [-0.0141],
        [-0.0002],
        [-0.0021],
        [-0.0123],
        [ 0.0023],
        [-0.0078]], dtype=torch.float64)